In [1]:
import numpy as np
import cv2
import os
import random
import torch
from torch.autograd import Variable
from torchvision import transforms
output_path = "mnist_train_images/"

/home/fest/anaconda3/envs/pythonProject/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/fest/anaconda3/envs/pythonProject/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
data = np.genfromtxt("mnist_train.csv", delimiter=',', skip_header=1)
labels = data[:, 0].astype(np.uint8)
values = data[:,1:].astype(np.uint8)
images = np.reshape(values, (-1,28,28))


In [9]:

# os.makedirs(output_path, exist_ok=True)
# for i in range(len(images)):
#     filename = os.path.join(output_path, f'{i}.jpg')
#     cv2.imwrite(filename, images[i])

In [34]:
# images1 = []
# images_path = [os.path.join(output_path, f) for f in os.listdir(output_path) if f.endswith('.jpg')]
# for i, path in enumerate(images_path):
#     temp = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#     if i != 60000:
#         images1.append(temp)

In [38]:
# images1 = np.asarray(images1)

In [3]:
learning_rate = 0.01
batch_size = 200
epochs = 10
log_interval = 10

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)


In [6]:
net = Net()
net

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)

In [7]:
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum=0.9)
criterion = nn.NLLLoss()

In [8]:
def rot_mat(deg):
   theta = deg/ 180 * np.pi
   c,s = np.cos(theta),np.sin(theta)
   return np.array([[c,-s],[s,c]])

In [9]:
def rotate(im):
    degree = random.randint(1,180)
    h, w, c = im.shape
    h2, w2 = h // 2, w // 2
    wr2, hr2 = (np.max(np.abs(rot_mat(degree) @ np.array([[-w2, w2, w2], [h2, h2, -h2]])), axis=1)).astype(np.int32)
    wr, hr = wr2 * 2, hr2 * 2
    imr = np.zeros((wr, hr, 3))
    yr, xr = np.indices((hr, wr))
    yr, xr = yr.flatten(), xr.flatten()
    yrc, xrc = yr - hr2, xr - wr2
    xc, yc = (rot_mat(-degree) @ np.row_stack((xrc, yrc))).astype(np.int32)
    x, y = xc + w2, yc + h2
    include = np.logical_and(np.abs(xc) < w2, np.abs(yc) < h2)
    include.mean()
    imr[yr[include], xr[include]] = im[y[include], x[include]]
    return imr

In [10]:
def noise(im):
    noize = np.zeros(im.shape,np.uint8)
    cv2.randn(noize,0,50)
    n_im = cv2.add(im,noize)
    return n_im

In [19]:
def normalize(im):
    normalized_image = cv2.normalize(im,None,0,255,cv2.NORM_MINMAX)
    normalized_image = np.uint8(normalized_image)
    return normalized_image

In [20]:
def tensor(im):
    tensor_image = np.expand_dims(im, axis=0)
    return tensor_image

In [32]:
normalized_images = normalize(images)
normalized_labels = normalize(labels)
normalized_images = np.array_split(normalized_images, len(normalized_images) // batch_size)
normalized_labels = np.array_split(normalized_labels, len(normalized_labels) // batch_size)

In [42]:

t_mini_batches = torch.Tensor(np.array(normalized_images))
t_label_batches = torch.from_numpy(np.array(normalized_labels))

In [43]:
for epoch in range(epochs):
    for batch in range(batch_size):
        data, target = Variable(t_mini_batches[batch], Variable(t_label_batches[batch]))
        optimizer.zero_grad()
        net_out = net(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()
        if batch % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch * len(data), len(len(images)),
                       100. * batch / len(images), loss.data[0]))

TypeError: only integer tensors of a single element can be converted to an index

In [37]:
type(t_mini_batches)
type(t_label_batches)

torch.Tensor